In [ ]:
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import  Convolution2D

## Learning Objective Error Map

In the first stage of training, the objective error maps are used as proxy regression targets to get the effect of 
increasing data. The loss function is defined by the mean squared error between the predicted and ground-truth error
maps.

\begin{align*}
\mathcal{L}_1(\hat{I}_d; \theta_f, \theta_g) = ||g(f(\hat{I}_d, \theta_f), \theta_g) - \mathbf{e}_{gt}) \odot \mathbf{\hat{r}}||^2_2
\end{align*}

where,

\begin{align*}
\mathbf{e}_{gt} = err(\hat{I}_r, \hat{I}_d)
\end{align*}

and $err(\cdot)$ is any error function. The authors decided to use

\begin{align*}
\mathbf{e}_{gt} = | \hat{I}_r -  \hat{I}_d | ^ p
\end{align*}

with $p=0.2$ in order to prevent that the values in the error map are small or close to zero.

In [ ]:
@tf.function
def error_map(reference: tf.Tensor, distorted: tf.Tensor, p: float=0.2) -> tf.Tensor:
    assert reference.shape == distorted.shape, 'Both images must be of the same size'
    return tf.pow(tf.abs(reference - distorted), p)

## Reliability Map Prediction

According to the author, the model is likely to fail to predict the objective error map of
homogeneous regions without having information of its pristine image. Thus, he proposes a 
reliability function. The assumption is that blurry regions have lower reliability than textured 
regions.

\begin{align*}
\mathbf{r} = \frac{2}{1 + exp(-\alpha|\hat{I}_d|)} - 1
\end{align*}

where α controls the saturation property of the reliability map. To assign sufficiently
large values to pixels with small values, the positive part of a sigmoid is used.

In [ ]:
@tf.function
def reliability_map(distorted: tf.Tensor, alpha: float) -> tf.Tensor:
    assert distorted.dtype == tf.float32, 'The Tensor must by of dtype tf.float32'
    return 2 / (1 + tf.exp(- alpha * tf.abs(distorted)))

Besides, to prevent the reliability map to directly affect the predicted score,
it is divided by its average

\begin{align*}
\mathbf{\hat{r}} = \frac{1}{\frac{1}{H_rW_r}\sum_{(i,j)}\mathbf{r}(i,j)}\mathbf{r}
\end{align*}

In [ ]:
@tf.function
def average_reliability_map(distorted: tf.Tensor, alpha: float) -> tf.Tensor:
    r = reliability_map(distorted, alpha)
    return 1 / tf.reduce_mean(r) * r

In [ ]:
model = Sequential([
    Convolution2D(48, (3, 3), name='Conv1', activation='relu', input_shape=(None, None, 1), padding='same', strides=(2, 2)),
    Convolution2D(48, (3, 3), name='Conv2', activation='relu', padding='same'),
    Convolution2D(64, (3, 3), name='Conv3', activation='relu', padding='same', strides=(2, 2)),
    Convolution2D(64, (3, 3), name='Conv4', activation='relu', padding='same'),
    Convolution2D(64, (3, 3), name='Conv5', activation='relu', padding='same'),
    Convolution2D(64, (3, 3), name='Conv6', activation='relu', padding='same'),
    Convolution2D(128, (3, 3), name='Conv7', activation='relu', padding='same'),
    Convolution2D(128, (3, 3), name='Conv8', activation='relu', padding='same'),
    Convolution2D(1, (1, 1), name='Conv9', padding='same'),
])

In [ ]:
optimizer = tf.optimizers.Nadam(learning_rate=2 * 10 ** -4)
model.compile(
    optimizer=optimizer,
    loss=tf.losses.MeanSquaredError(),
    metrics=[tf.metrics.Accuracy()])

In [ ]:
model.summary()